We aim to uncover the insider stock trades of Michael Saylor, MicroStrategy’s Executive Chairman, whose company has turned into a de facto Bitcoin treasury.

In [2]:
pip install sec-edgar-downloader pandas lxml tqdm

Note: you may need to restart the kernel to use updated packages.


In [39]:
### Our first goal is to download all Form 4 filings associated with Michael J. Saylor and, for thoroughness, MicroStrategy. 

from sec_edgar_downloader import Downloader
import os

### Let's initialize the downloader, using your email and desired downloads folder
dl = Downloader("downloads", "juanlasso65@gmail.com")

### Define the CIK, the SEC’s unique identifier, along with the form type.
ciks = {
    "MicroStrategy (company)": "0001050446",
    "Michael J. Saylor (personal)": "0001079782"
}
form_type = "4"


### Download all Form 4s

for name, cik in ciks.items():
    print(f"Downloading Form {form_type} filings for {name} (CIK {cik})...")
    dl.get(form_type, cik)

### Let's count the downloaded filings
base_path = "downloads/sec-edgar-filings"

for name, cik in ciks.items():
    form4_path = os.path.join(base_path, cik, form_type)
    if os.path.exists(form4_path):
        num_files = len([f for f in os.listdir(form4_path) if os.path.isfile(os.path.join(form4_path, f))])
        print(f"{name} (CIK {cik}) has {num_files} Form 4 filings downloaded.")
    else:
        print(f"No Form 4 filings found for {name} (CIK {cik})")

No Form 4 filings found for MicroStrategy (company) (CIK 0001050446)
No Form 4 filings found for Michael J. Saylor (personal) (CIK 0001079782)


In [31]:
## Let's check the number of files 
from sec_edgar_downloader import Downloader

dl = Downloader("downloads", "juanlasso65@gmail.com")
dl.get("4", "0001050446")

629

In [17]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

BASE_DIR = "sec-edgar-filings/0001079782/4"

def parse_form4_directory(base_dir=BASE_DIR):
### First let's extract the </XML> block text, which contains the data we want from the larger Form 4 text.
    def extract_xml_block(text):
    
        match = re.search(r"<XML>(.*?)</XML>", text, flags=re.DOTALL)
        return match.group(1) if match else None

    def to_float(x):
        try:
            return float(x)
        except:
            return None

    def get_nested(tx, path):
### This is a helper AI recommended to safely traverse nested XML tags.
        node = tx
        for p in path:
            node = node.find(p)
            if node is None:
                return None
        return node.text.strip() if node else None

    def parse_transactions(xml_str, file_path):
        soup = BeautifulSoup(xml_str, "xml")
        results = []

### Get ticker symbol once per file
        symbol = soup.find("issuerTradingSymbol")
        ticker = symbol.text.strip() if symbol else None

### Let's do unified parsing for both non-derivative and derivative transactions
        tx_nodes = soup.find_all(["nonDerivativeTransaction", "derivativeTransaction"])

        for tx in tx_nodes:
            results.append({
                "date": get_nested(tx, ["transactionDate", "value"]),
                "ticker": ticker,
                "code": get_nested(tx, ["transactionCoding", "transactionCode"]),
                "shares": to_float(get_nested(tx, ["transactionAmounts", "transactionShares", "value"])),
                "price": to_float(get_nested(tx, ["transactionAmounts", "transactionPricePerShare", "value"])),
                "acquire_dispose": get_nested(tx, ["transactionAmounts", "transactionAcquiredDisposedCode", "value"]),
                "post_transaction_shares": to_float(get_nested(tx, ["postTransactionAmounts", "sharesOwnedFollowingTransaction", "value"])),
                "ownership": get_nested(tx, ["ownershipNature", "directOrIndirectOwnership", "value"]),
                "class_type": get_nested(tx, ["securityTitle", "value"]),
                "file_path": file_path
            })

        return results

### Now let's apply this to the main loop 
    
    all_rows = []

    for root, dirs, files in os.walk(base_dir):
        for f in files:
            if not f.lower().endswith(".txt"):
                continue
            file_path = os.path.join(root, f)
            with open(file_path, "r", encoding="utf-8", errors="ignore") as fp:
                text = fp.read()

            xml_block = extract_xml_block(text)
            if not xml_block:
                continue

            rows = parse_transactions(xml_block, file_path)
            all_rows.extend(rows)

    df = pd.DataFrame(all_rows)

    # Ensure proper types
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['shares'] = pd.to_numeric(df['shares'], errors='coerce')
    df['year'] = df['date'].dt.year

    return df

df = parse_form4_directory()
df

,date,ticker,code,shares,price,acquire_dispose,post_transaction_shares,ownership,class_type,file_path,year
0,2024-04-17,MSTR,M,5000.0,121.43,A,5000.0,D,Class A Common Stock,sec-edgar-filings/0001079782/4/0000950170-24-0...,2024
1,2024-04-17,MSTR,S,24.0,1140.79,D,4976.0,D,Class A Common Stock,sec-edgar-filings/0001079782/4/0000950170-24-0...,2024
2,2024-04-17,MSTR,S,41.0,1144.44,D,4935.0,D,Class A Common Stock,sec-edgar-filings/0001079782/4/0000950170-24-0...,2024
3,2024-04-17,MSTR,S,34.0,1146.75,D,4901.0,D,Class A Common Stock,sec-edgar-filings/0001079782/4/0000950170-24-0...,2024
4,2024-04-17,MSTR,S,43.0,1147.05,D,4858.0,D,Class A Common Stock,sec-edgar-filings/0001079782/4/0000950170-24-0...,2024
...,...,...,...,...,...,...,...,...,...,...,...
4207,2009-12-01,MSTR,S,100.0,89.61,D,5195.0,I,Class A Common Stock,sec-edgar-filings/0001079782/4/0001079782-09-0...,2009
4208,2009-12-01,MSTR,S,2100.0,89.65,D,3095.0,I,Class A Common Stock,sec-edgar-filings/0001079782/4/0001079782-09-0...,2009
4209,2009-12-01,MSTR,S,200.0,89.66,D,2895.0,I,Class A Common Stock,sec-edgar-filings/0001079782/4/0001079782-09-0...,2009
4210,2009-12-01,MSTR,S,2895.0,89.70,D,0.0,I,Class A Common Stock,sec-edgar-filings/0001079782/4/0001079782-09-0...,2009


In [46]:
import pandas as pd

# Filter for Class B Common Stock
class_b_tx = df[df['class_type'] == 'Class B Common Stock'].copy()


class_b_tx['date'] = pd.to_datetime(class_b_tx['date'], errors='coerce')
class_b_tx['shares'] = pd.to_numeric(class_b_tx['shares'], errors='coerce')
class_b_tx['year'] = class_b_tx['date'].dt.year

# Only keep rows with a valid year and non-null shares
class_b_tx = class_b_tx[class_b_tx['year'].notnull() & class_b_tx['shares'].notnull()]

# Some transactions might not use 'A' or 'D' in acquire_dispose, so we'll treat 'A' as acquired, 'D' as disposed; everything else we can ignore or log
class_b_tx['shares_acquired'] = class_b_tx.apply(lambda x: x['shares'] if x['acquire_dispose'] == 'A' else 0, axis=1)
class_b_tx['shares_disposed'] = class_b_tx.apply(lambda x: x['shares'] if x['acquire_dispose'] == 'D' else 0, axis=1)

# Aggregate by year
annual_class_b = class_b_tx.groupby('year').agg(
    total_shares_acquired=pd.NamedAgg(column='shares_acquired', aggfunc='sum'),
    total_shares_disposed=pd.NamedAgg(column='shares_disposed', aggfunc='sum')
).reset_index()

# Compute net change
annual_class_b['net_shares_change'] = annual_class_b['total_shares_acquired'] - annual_class_b['total_shares_disposed']

# Sort by year
annual_class_b = annual_class_b.sort_values('year').reset_index(drop=True)

# Display
print("Annual Class B Stock Transaction")
print(annual_class_b)

Annual Class B Stock Transaction
   year  total_shares_acquired  total_shares_disposed  net_shares_change
0  2005                      0                60270.0           -60270.0
1  2006                      0               359848.0          -359848.0
2  2009                      0                70882.0           -70882.0
3  2011                      0               301532.0          -301532.0
4  2012                      0                45500.0           -45500.0
5  2020                      0                50000.0           -50000.0


In [48]:
import pandas as pd

# Filter for Class A Common Stock
class_a_tx = df[df['class_type'] == 'Class A Common Stock'].copy()

# Ensure proper types
class_a_tx['date'] = pd.to_datetime(class_a_tx['date'], errors='coerce')
class_a_tx['shares'] = pd.to_numeric(class_a_tx['shares'], errors='coerce')
class_a_tx['year'] = class_a_tx['date'].dt.year

# Only keep rows with valid year and shares
class_a_tx = class_a_tx[class_a_tx['year'].notnull() & class_a_tx['shares'].notnull()]

# Map transaction codes: 'A' = acquired, 'D' = disposed
class_a_tx['shares_acquired'] = class_a_tx.apply(lambda x: x['shares'] if x['acquire_dispose'] == 'A' else 0, axis=1)
class_a_tx['shares_disposed'] = class_a_tx.apply(lambda x: x['shares'] if x['acquire_dispose'] == 'D' else 0, axis=1)

# Aggregate by year
annual_class_a = class_a_tx.groupby('year').agg(
    total_shares_acquired=pd.NamedAgg(column='shares_acquired', aggfunc='sum'),
    total_shares_disposed=pd.NamedAgg(column='shares_disposed', aggfunc='sum')
).reset_index()

# Compute net change
annual_class_a['net_shares_change'] = annual_class_a['total_shares_acquired'] - annual_class_a['total_shares_disposed']

# Sort by year
annual_class_a = annual_class_a.sort_values('year').reset_index(drop=True)

# Display
print("Annual Class A Stock Transactions")
print(annual_class_a)

Annual Class A Stock Transactions
   year  total_shares_acquired  total_shares_disposed  net_shares_change
0  2004                10200.0                10200.0                0.0
1  2005                60270.0                61152.0             -882.0
2  2006               359848.0               359848.0                0.0
3  2009                77536.0                77536.0                0.0
4  2011               301532.0               301532.0                0.0
5  2012               438646.0               438646.0                0.0
6  2020                50000.0                50000.0                0.0
7  2024               400000.0               399900.0              100.0
